## Preprocessing ##
- load library

In [1]:
import re
import string
import time
from copy import deepcopy

- load dataset

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('review_mpaspor.csv')

In [4]:
df.head()

,userName,score,at,content
0,Anggun Virly channel,1,2024-01-05 22:03:03,Kode billing tidak ada
1,Juni Arjuna,1,2024-01-05 16:47:02,"App nya saya mohon di perbaiki,saya udh log-in..."
2,Khamdan Ambari,1,2024-01-05 15:54:11,"Maksudnya apa ini qplikasi, kirain bisa memban..."
3,Antika Yosa Fitri,1,2024-01-05 15:14:19,"Bikin aplikasi, bukan memudahkan tetapi bikin ..."
4,PENDAKI PAM chanel,1,2024-01-05 11:15:01,aplikasi makan sagu Rakyat miskin


In [5]:
df = df.drop(['score','at'], axis=1)

In [6]:
df

,userName,content
0,Anggun Virly channel,Kode billing tidak ada
1,Juni Arjuna,"App nya saya mohon di perbaiki,saya udh log-in..."
2,Khamdan Ambari,"Maksudnya apa ini qplikasi, kirain bisa memban..."
3,Antika Yosa Fitri,"Bikin aplikasi, bukan memudahkan tetapi bikin ..."
4,PENDAKI PAM chanel,aplikasi makan sagu Rakyat miskin
...,...,...
12494,Sukirno Nur,Mantabbbb
12495,Ifan Sastra Hirata,Tidak ditemukan Kanim di lokasi anda
12496,Handriyanti,Lemot bgt pas milih lokasi pengambilan paspor ...
12497,Ibnu Prayudha Pangestu,Aplikasi ini sangat membantu bagi saya yang me...


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12499 entries, 0 to 12498
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   userName  12499 non-null  object
 1   content   12499 non-null  object
dtypes: object(2)
memory usage: 195.4+ KB


- Preprocessing dataset

In [8]:
#!pip install ekphrasis

In [9]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['email', 'percent', 'money', 'phone', 'user',
        'time', 'date', 'number'],
    # terms that will be annotated
    #annotate={"hashtag", "allcaps", "elongated", "repeated",'emphasis', 'censored'},
    annotate={"hashtag"},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words

    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

C:\Users\62895\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ekphrasis\classes\tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading twitter - 1grams ...


C:\Users\62895\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ekphrasis\classes\exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [10]:
# panggil ekphrasis

def bersih_data(text):
    return " ".join(text_processor.pre_process_doc(text))

In [11]:
# fungsi dari AMS 01-03. silakan cek bagaimana saya merubah menjadi fungsi

def non_ascii(text):
    return text.encode('ascii', 'replace').decode('ascii')

def remove_space_alzami(text):
    return " ".join(text.split())

def remove_emoji_alzami(text):
    return ' '.join(re.sub("([x#][A-Za-z0-9]+)"," ", text).split())

def remove_tab(text):
    return text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")

def remove_tab2(text):
    return re.sub('\s+',' ',text)

def remove_rt(text):
    return text.replace('RT'," ")

def remove_mention(text):
    return ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())

def remove_incomplete_url(text):
    return text.replace("http://", " ").replace("https://", " ")

def remove_single_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

def remove_excessive_dot(text):
    return text.replace('..'," ")

def change_stripe(text):
    return text.replace('-'," ")

def lower(text):
    return text.lower()

def remove_single_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

def remove_excessive_dot(text):
    return text.replace('..'," ")

def lower(text):
    return text.lower()

def remove_whitespace_LT(text):
    return text.strip()

def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

def remove_punctuation(text):
    remove = string.punctuation
    remove = remove.replace("_", "") # don't remove hyphens
    pattern = r"[{}]".format(remove) # create the pattern
    return re.sub(pattern, "", text) 

In [12]:
# hapus untuk <>
def remove_number_eks(text):
    return text.replace('<number>'," ")

def remove_angka(text):
    return re.sub(r"\d+", "", text) 

def remove_URL_eks(text):
    return text.replace('URL'," ").replace('url'," ")

def space_punctuation(text):
    return re.sub('(?<! )(?=[.,!?()])|(?<=[.,!?()])(?! )', r' ', text)

- lakukan pembersihan dengan memanggil fungsi yang didefinnisikan diatas

In [13]:
i = 0
final_string = []
s = ""
for text in df['content'].values:
    filteredSentence = []
    EachReviewText = ""
    proc = remove_rt(text)
    proc = lower(proc)
    proc = change_stripe(proc)
    proc = remove_emoji_alzami(proc)
    proc = remove_tab(proc)
    proc = remove_tab2(proc)
    proc = non_ascii(proc)
    proc = remove_incomplete_url(proc)
    proc = remove_excessive_dot(proc)
    proc = remove_whitespace_LT(proc)
    proc = remove_whitespace_multiple(proc)
    proc = remove_single_char(proc)
    proc = space_punctuation(proc)
    proc = remove_punctuation(proc)
    proc = remove_space_alzami(proc)
    proc = bersih_data(proc)
    proc = remove_number_eks(proc)
    proc = remove_angka(proc) 
    proc = remove_URL_eks(proc)
    EachReviewText = proc
    final_string.append(EachReviewText)

In [14]:
df["step01"] = final_string

In [15]:
df.head(10)

,userName,content,step01
0,Anggun Virly channel,Kode billing tidak ada,kode billing tidak ada
1,Juni Arjuna,"App nya saya mohon di perbaiki,saya udh log-in...",app nya saya mohon di perbaiki saya udh log in...
2,Khamdan Ambari,"Maksudnya apa ini qplikasi, kirain bisa memban...",maksudnya apa ini qplikasi kirain bisa membant...
3,Antika Yosa Fitri,"Bikin aplikasi, bukan memudahkan tetapi bikin ...",bikin aplikasi bukan memudahkan tetapi bikin t...
4,PENDAKI PAM chanel,aplikasi makan sagu Rakyat miskin,aplikasi makan sagu rakyat miskin
5,Nora Wiqayati,"Paspor nggak bisa dinyatakan rusak apa tidak, ...",paspor nggak bisa dinyatakan rusak apa tidak p...
6,Rendy Sihaloho,Ini gimana si di pilih tanggal gak mau gabisa ...,ini gimana si di pilih tanggal gak mau gabisa ...
7,tommy fondy,Good,good
8,abaz Gorden,"Aplikasi milik negara kok gini amat ya, susah ...",aplikasi milik negara kok gini amat ya susah b...
9,EDY MUSTOFA,Aplikasinya Tidak bisa dibuka,aplikasinya tidak bisa dibuka


- hapus data kosong

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12499 entries, 0 to 12498
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   userName  12499 non-null  object
 1   content   12499 non-null  object
 2   step01    12499 non-null  object
dtypes: object(3)
memory usage: 293.1+ KB


In [17]:
df_hapus = df[~df['step01'].str.contains(" ")]

In [18]:
df_hapus.info()

<class 'pandas.core.frame.DataFrame'>
Index: 811 entries, 7 to 12494
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   userName  811 non-null    object
 1   content   811 non-null    object
 2   step01    811 non-null    object
dtypes: object(3)
memory usage: 25.3+ KB


In [19]:
df_hapus.head(10)

,userName,content,step01
7,tommy fondy,Good,good
21,Ichsan Maulana,Mantanance?,mantanance
66,Sugiarto Rudi,Tooop,tooop
100,warkima03 warkima03,ok,ok
109,elvan dika,mantap,mantap
111,Edi Rudi,Good,good
116,Siti Hajar,Good,good
147,astika yunita,Aneh,aneh
149,uphie noer,Good,good
174,Number two,Buruk,buruk


In [20]:
df_new = df[~df.isin(df_hapus)].dropna()

In [21]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11688 entries, 0 to 12498
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   userName  11688 non-null  object
 1   content   11688 non-null  object
 2   step01    11688 non-null  object
dtypes: object(3)
memory usage: 365.2+ KB


In [22]:
df_new

,userName,content,step01
0,Anggun Virly channel,Kode billing tidak ada,kode billing tidak ada
1,Juni Arjuna,"App nya saya mohon di perbaiki,saya udh log-in...",app nya saya mohon di perbaiki saya udh log in...
2,Khamdan Ambari,"Maksudnya apa ini qplikasi, kirain bisa memban...",maksudnya apa ini qplikasi kirain bisa membant...
3,Antika Yosa Fitri,"Bikin aplikasi, bukan memudahkan tetapi bikin ...",bikin aplikasi bukan memudahkan tetapi bikin t...
4,PENDAKI PAM chanel,aplikasi makan sagu Rakyat miskin,aplikasi makan sagu rakyat miskin
...,...,...,...
12493,Muhamad Dikri,Keren Banget.. gampang banget prosesnya.. good...,keren banget gampang banget prosesnya good job
12495,Ifan Sastra Hirata,Tidak ditemukan Kanim di lokasi anda,tidak ditemukan kanim di lokasi anda
12496,Handriyanti,Lemot bgt pas milih lokasi pengambilan paspor ...,lemot bgt pas milih lokasi pengambilan paspor ...
12497,Ibnu Prayudha Pangestu,Aplikasi ini sangat membantu bagi saya yang me...,aplikasi ini sangat membantu bagi saya yang me...


- normalisasi kata slang menjadi kata baku

In [23]:
# token
import nltk
from nltk.tokenize import word_tokenize 

In [24]:
def word_tokenize_wrapper(text):
  return word_tokenize(text)

In [25]:
df_new['tokens'] = df['step01'].apply(word_tokenize_wrapper)

In [26]:
df_new.head(10)

,userName,content,step01,tokens
0,Anggun Virly channel,Kode billing tidak ada,kode billing tidak ada,"[kode, billing, tidak, ada]"
1,Juni Arjuna,"App nya saya mohon di perbaiki,saya udh log-in...",app nya saya mohon di perbaiki saya udh log in...,"[app, nya, saya, mohon, di, perbaiki, saya, ud..."
2,Khamdan Ambari,"Maksudnya apa ini qplikasi, kirain bisa memban...",maksudnya apa ini qplikasi kirain bisa membant...,"[maksudnya, apa, ini, qplikasi, kirain, bisa, ..."
3,Antika Yosa Fitri,"Bikin aplikasi, bukan memudahkan tetapi bikin ...",bikin aplikasi bukan memudahkan tetapi bikin t...,"[bikin, aplikasi, bukan, memudahkan, tetapi, b..."
4,PENDAKI PAM chanel,aplikasi makan sagu Rakyat miskin,aplikasi makan sagu rakyat miskin,"[aplikasi, makan, sagu, rakyat, miskin]"
5,Nora Wiqayati,"Paspor nggak bisa dinyatakan rusak apa tidak, ...",paspor nggak bisa dinyatakan rusak apa tidak p...,"[paspor, nggak, bisa, dinyatakan, rusak, apa, ..."
6,Rendy Sihaloho,Ini gimana si di pilih tanggal gak mau gabisa ...,ini gimana si di pilih tanggal gak mau gabisa ...,"[ini, gimana, si, di, pilih, tanggal, gak, mau..."
8,abaz Gorden,"Aplikasi milik negara kok gini amat ya, susah ...",aplikasi milik negara kok gini amat ya susah b...,"[aplikasi, milik, negara, kok, gini, amat, ya,..."
9,EDY MUSTOFA,Aplikasinya Tidak bisa dibuka,aplikasinya tidak bisa dibuka,"[aplikasinya, tidak, bisa, dibuka]"
10,Dian prahmana putra,"Aneh banget ya masa kuota penuh terus,jadi mau...",aneh banget ya masa kuota penuh terus jadi mau...,"[aneh, banget, ya, masa, kuota, penuh, terus, ..."


In [27]:
normalized_word = pd.read_csv('kamus_clean.csv')

In [28]:
normalized_word_dict = {}

for index, row in normalized_word.iterrows():
    if row[0] not in normalized_word_dict:
        normalized_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalized_word_dict[term] if term in normalized_word_dict else term for term in document]

C:\Users\62895\AppData\Local\Temp\ipykernel_6732\2408214049.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[0] not in normalized_word_dict:
C:\Users\62895\AppData\Local\Temp\ipykernel_6732\2408214049.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  normalized_word_dict[row[0]] = row[1]


In [29]:
df_new['final_tokens'] = df_new['tokens'].apply(normalized_term)

In [30]:
i=0
final_string_tokens = []
for text in df_new['final_tokens'].values:
    EachReviewText = ""
    EachReviewText = ' '.join(text)
    final_string_tokens.append(EachReviewText)

In [31]:
df_new["step02"] = final_string_tokens

In [32]:
df_new.head(10)

,userName,content,step01,tokens,final_tokens,step02
0,Anggun Virly channel,Kode billing tidak ada,kode billing tidak ada,"[kode, billing, tidak, ada]","[kode, billing, tidak, ada]",kode billing tidak ada
1,Juni Arjuna,"App nya saya mohon di perbaiki,saya udh log-in...",app nya saya mohon di perbaiki saya udh log in...,"[app, nya, saya, mohon, di, perbaiki, saya, ud...","[app, nya, saya, mohon, di, perbaiki, saya, ud...",app nya saya mohon di perbaiki saya udh log in...
2,Khamdan Ambari,"Maksudnya apa ini qplikasi, kirain bisa memban...",maksudnya apa ini qplikasi kirain bisa membant...,"[maksudnya, apa, ini, qplikasi, kirain, bisa, ...","[maksudnya, apa, ini, qplikasi, kirain, bisa, ...",maksudnya apa ini qplikasi kirain bisa membant...
3,Antika Yosa Fitri,"Bikin aplikasi, bukan memudahkan tetapi bikin ...",bikin aplikasi bukan memudahkan tetapi bikin t...,"[bikin, aplikasi, bukan, memudahkan, tetapi, b...","[bikin, aplikasi, bukan, memudahkan, tetapi, b...",bikin aplikasi bukan memudahkan tetapi bikin t...
4,PENDAKI PAM chanel,aplikasi makan sagu Rakyat miskin,aplikasi makan sagu rakyat miskin,"[aplikasi, makan, sagu, rakyat, miskin]","[aplikasi, makan, sagu, rakyat, miskin]",aplikasi makan sagu rakyat miskin
5,Nora Wiqayati,"Paspor nggak bisa dinyatakan rusak apa tidak, ...",paspor nggak bisa dinyatakan rusak apa tidak p...,"[paspor, nggak, bisa, dinyatakan, rusak, apa, ...","[paspor, nggak, bisa, dinyatakan, rusak, apa, ...",paspor nggak bisa dinyatakan rusak apa tidak p...
6,Rendy Sihaloho,Ini gimana si di pilih tanggal gak mau gabisa ...,ini gimana si di pilih tanggal gak mau gabisa ...,"[ini, gimana, si, di, pilih, tanggal, gak, mau...","[ini, gimana, si, di, pilih, tanggal, gak, mau...",ini gimana si di pilih tanggal gak mau gabisa ...
8,abaz Gorden,"Aplikasi milik negara kok gini amat ya, susah ...",aplikasi milik negara kok gini amat ya susah b...,"[aplikasi, milik, negara, kok, gini, amat, ya,...","[aplikasi, milik, negara, kok, gini, amat, ya,...",aplikasi milik negara kok gini amat ya susah b...
9,EDY MUSTOFA,Aplikasinya Tidak bisa dibuka,aplikasinya tidak bisa dibuka,"[aplikasinya, tidak, bisa, dibuka]","[aplikasinya, tidak, bisa, dibuka]",aplikasinya tidak bisa dibuka
10,Dian prahmana putra,"Aneh banget ya masa kuota penuh terus,jadi mau...",aneh banget ya masa kuota penuh terus jadi mau...,"[aneh, banget, ya, masa, kuota, penuh, terus, ...","[aneh, banget, ya, masa, kuota, penuh, terus, ...",aneh banget ya masa kuota penuh terus jadi mau...


## simpan

In [33]:
df_new.to_csv('clean_dataset.csv',sep=";")